##### Author: Praveen Saxena
##### Email: saxep01@gmail.com
##### Create Date: 7/14/2021
##### Purpose: Explore and prep data for analysis

In [1]:
import pandas as pd
from pprint import pprint
from IPython.display import display, Markdown


In [2]:
%%capture 

from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
nanohub_metrics_db = application.new_db_engine('nanohub_metrics')

## Total Users

In [4]:

sql = '''
SELECT COUNT(*) AS `Total Users`
    FROM nanohub.jos_users
;
'''

df = pd.read_sql(sql, nanohub_db)
df

,Total Users
0,253583


## One Day Users

In [5]:

sql = '''
SELECT COUNT(*) AS `One Day Users`
    FROM nanohub.jos_users
    WHERE registerDate > DATE_SUB(lastvisitDate, INTERVAL 1 DAY)
;
'''

df = pd.read_sql(sql, nanohub_metrics_db)
df

,One Day Users
0,118932


## Users Who Never Logged-In Again Since Registration

In [8]:

sql = '''
SELECT COUNT(*) AS `Single Login Users`
    FROM nanohub.jos_users
    WHERE registerDate = DATE_SUB(lastvisitDate, INTERVAL 1 DAY)
;
'''

df = pd.read_sql(sql, nanohub_db)
df

,Single Login Users
0,1


# RECENCY

In [9]:

sql = '''
SELECT DISTINCT jos_users.id, jos_users.name, jos_users.registerDate, jos_users.lastvisitDate,  nanohub_metrics.userlogin.datetime
    FROM nanohub.jos_users
    JOIN (
            SELECT MAX(datetime) max_datetime, uidnumber, id, user, action
                FROM nanohub_metrics.userlogin
                WHERE action = 'login'
            GROUP BY uidnumber
        ) u_max ON (u_max.uidnumber = jos_users.id)
    JOIN nanohub_metrics.userlogin userlogin ON (userlogin.datetime = u_max.max_datetime)
    LIMIT 10
;
'''

df = pd.read_sql(sql, nanohub_db)
df

,id,name,registerDate,lastvisitDate,datetime
0,1000,CMS Admin Manager,NaT,NaT,2009-02-17 16:15:10
1,1684,Grid Statistics,2008-11-18 17:29:56,2020-02-14 18:50:14,2020-02-14 18:50:14
2,1735,Colombo R. Bolognesi,2007-01-29 09:34:45,2019-03-15 09:00:31,2019-03-15 05:00:31
3,1742,H.-S. Philip Wong,2005-07-28 01:28:13,2015-02-18 17:03:45,2015-02-18 12:03:20
4,1758,Abdul Majid,2002-04-17 09:43:44,2010-06-02 20:22:00,2010-06-02 16:22:00
5,1767,Osama Munir Nayfeh,2005-07-29 18:24:36,2012-10-02 17:22:20,2012-10-02 13:22:20
6,1768,Max Howk,2007-06-27 23:44:22,2021-05-22 05:29:00,2021-05-22 01:22:45
7,1790,Gijs Bosman,2003-05-30 17:30:36,2012-08-30 16:09:46,2012-08-30 12:09:46
8,1794,Mark Ratner,2006-07-04 19:08:00,2010-08-17 14:19:36,2010-08-17 10:16:34
9,1815,Gerhard Goldbeck-Wood,2005-02-10 07:50:17,2010-10-04 16:13:47,2010-10-04 12:13:47


In [10]:
sql = '''
EXPLAIN SELECT DISTINCT jos_users.id, jos_users.name, jos_users.registerDate, jos_users.lastvisitDate,  nanohub_metrics.userlogin.datetime
    FROM nanohub.jos_users
    JOIN (
            SELECT MAX(datetime) max_datetime, uidnumber, id, user, action
                FROM nanohub_metrics.userlogin
                WHERE action = 'login'
            GROUP BY uidnumber
        ) u_max ON (u_max.uidnumber = jos_users.id)
    JOIN nanohub_metrics.userlogin userlogin ON (userlogin.datetime = u_max.max_datetime)
    LIMIT 10
;
'''

df = pd.read_sql(sql, nanohub_db)
df

,id,select_type,table,type,possible_keys,key,key_len,ref,rows,Extra
0,1,PRIMARY,jos_users,ALL,PRIMARY,None,None,None,263425,Using temporary
1,1,PRIMARY,<derived2>,ref,key0,key0,9,nanohub.jos_users.id,1840,Using where
2,1,PRIMARY,userlogin,ref,userlogin,userlogin,8,u_max.max_datetime,2,Using index
3,2,DERIVED,userlogin,ALL,None,None,None,None,484906592,Using where; Using temporary; Using filesort


# FREQUENCY

In [7]:

# sql = '''
# SELECT users.id, users.name, users.registerDate, COUNT(action) AS `num_actions`
#     FROM nanohub.jos_users AS users
#     LEFT JOIN nanohub_metrics.userlogin as logins ON logins.uidnumber = users.id
#     WHERE logins.uidnumber != 0 && logins.uidnumber IS NOT NULL
#     LIMIT 1
# ;
# '''

# df = pd.read_sql(sql, nanohub_db)
# df

# FREQUENCY 2

In [14]:

sql = '''
SELECT DISTINCT users.id, users.name, users.registerDate,
        COUNT(action) AS `num_actions`, COUNT(DISTINCT DATE(datetime)) as `distinct_days`,
        COUNT(DISTINCT DATE(datetime))/COUNT(action) as Frequency
    FROM nanohub.jos_users AS users
        LEFT JOIN nanohub_metrics.userlogin as logins ON logins.uidnumber = users.id
        WHERE logins.uidnumber != 0 && logins.uidnumber IS NOT NULL
LIMIT 10
;
'''

df = pd.read_sql(sql, nanohub_db)
df

,id,name,registerDate,num_actions,distinct_days,Frequency
0,12434,Nicholas J Kisseberth,2006-03-02 11:48:20,2095033,4555,0.0022


In [11]:

sql = '''
EXPLAIN SELECT DISTINCT users.id, users.name, users.registerDate,
        COUNT(action) AS `num_actions`, COUNT(DISTINCT DATE(datetime)) as `distinct_days`,
        COUNT(DISTINCT DATE(datetime))/COUNT(action) as Frequency
    FROM nanohub.jos_users AS users
        LEFT JOIN nanohub_metrics.userlogin as logins ON logins.uidnumber = users.id
        WHERE logins.uidnumber != 0 && logins.uidnumber IS NOT NULL
LIMIT 10
;
'''

df = pd.read_sql(sql, nanohub_db)
df

,id,select_type,table,type,possible_keys,key,key_len,ref,rows,Extra
0,1,SIMPLE,logins,index,None,userlogin,951,None,484906592,Using where; Using index
1,1,SIMPLE,users,eq_ref,PRIMARY,PRIMARY,4,nanohub_metrics.logins.uidnumber,1,Using where


In [15]:

sql = '''
EXPLAIN SELECT DISTINCT users.id, users.name, users.registerDate,
        COUNT(action) AS `num_actions`, COUNT(DISTINCT DATE(datetime)) as `distinct_days`,
        COUNT(DISTINCT DATE(datetime))/COUNT(action) as Frequency
    FROM nanohub.jos_users AS users
        LEFT JOIN nanohub_metrics.userlogin as logins ON logins.uidnumber = users.id
LIMIT 1000
;
'''

df = pd.read_sql(sql, nanohub_db)
df


,id,select_type,table,type,possible_keys,key,key_len,ref,rows,Extra
0,1,SIMPLE,users,ALL,None,None,None,None,263425,
1,1,SIMPLE,logins,index,None,userlogin,951,None,484906592,Using where; Using index; Using join buffer (f...


Running _EXPLAIN_ on the queries above shows us the need for certain indexes. 
Examples:
- _nanohub_metrics.userlogin.uidnumber_
- _nanohub_metrics.userlogin.datetime_
- _nanohub.jos_users.registerDate_ 
- _nanohub.jos_users.lastvisitDate_ 


Another point of note is that many early users do not have a user id (_uidnumber_) at all.
So we try switching to usernames.

In [24]:
sql = '''
EXPLAIN SELECT id, username, registerDate, lastvisitDate, name, email, usertype
    FROM nanohub.jos_users
    WHERE registerDate <= DATE_SUB(lastvisitDate, INTERVAL 1 DAY)
;
'''

df = pd.read_sql(sql, nanohub_db)
df

,id,select_type,table,type,possible_keys,key,key_len,ref,rows,Extra
0,1,SIMPLE,jos_users,ALL,None,None,None,None,263425,Using where


Still need index on _nanohub.jos_users.registerDate_, and _nanohub.jos_users.lastvisitDate_.

In [3]:

sql = '''
EXPLAIN SELECT DISTINCT jos_users.id, jos_users.username, jos_users.name, jos_users.registerDate, jos_users.lastvisitDate, jos_users.email, jos_users.usertype, nanohub_metrics.userlogin.datetime
    FROM nanohub.jos_users
    JOIN (
            SELECT MAX(datetime) max_datetime, uidnumber, id, user, action
                FROM nanohub_metrics.userlogin
                WHERE action = 'login'
            GROUP BY user
        ) u_max ON (u_max.user = jos_users.username)
    JOIN nanohub_metrics.userlogin userlogin ON (userlogin.datetime = u_max.max_datetime)
    LIMIT 10
;
'''

df = pd.read_sql(sql, nanohub_db)
df

,id,select_type,table,type,possible_keys,key,key_len,ref,rows,Extra
0,1,PRIMARY,jos_users,ALL,"uidx_username,username",None,None,None,263425,Using temporary
1,1,PRIMARY,<derived2>,ref,key0,key0,767,nanohub.jos_users.username,1840,Using where
2,1,PRIMARY,userlogin,ref,userlogin,userlogin,8,u_max.max_datetime,2,Using index
3,2,DERIVED,userlogin,ALL,None,None,None,None,484906592,Using where; Using temporary; Using filesort
